In [1]:
# convert image links to colab image links

def image_linker(link):

    new_link = 'https://drive.google.com/uc?id='
    new_link = new_link + link.split('/')[-2]
    paste_link = '![](' + new_link + ')'
    return paste_link

In [2]:
image_linker('https://drive.google.com/file/d/1fuHA4l_WDy1RyVuWdLIoBMWQb8u3OBp6/view?usp=sharing')

'![](https://drive.google.com/uc?id=1fuHA4l_WDy1RyVuWdLIoBMWQb8u3OBp6)'

![](https://drive.google.com/uc?id=1nIIH9fBeKBcwzv0zK3T436iJlu8LhUg7)

![](https://drive.google.com/uc?id=1fuHA4l_WDy1RyVuWdLIoBMWQb8u3OBp6)

μ(k) be the mean for the random variable for each class y

(σk)^2 be the variance for the variable for each class

## Algorithm:

Posterior = (Likelihood x Prior) / Marginal

P(Y/X1,X2) = (P(X1/Y) x P(X2/Y) x P(Y)) / (P(X1) x P(X2))

Likelihood is also called Class Conditional Probability => (P(Xi/Y))

Prior probability of Y => P(Y)

Marginal or Prior probability of Xi => P(Xi)

- Take dataset and split into dependent and independent.

- Initialize weights and bias to 0.

- For each epoch, find ypred, ypred_prob, dw, db and update weight and bias.

- Continue running epochs, until loss function becomes stagnant.

- Predict values of test set and evaluate.

In [3]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

## Dataset

In [4]:
X, y = datasets.make_classification(n_samples=100, n_features=10, n_classes=2, weights=[0.7,0.3], random_state=4)

In [5]:
X.shape, y.shape

((100, 10), (100,))

In [6]:
values, counts = np.unique(y, return_counts=True)
values, counts

(array([0, 1]), array([70, 30]))

In [7]:
np.unique(y)

array([0, 1])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((80, 10), (20, 10), (80,), (20,))

In [10]:
X_train[:5]

array([[-0.09634524,  1.19336851, -0.26937493, -0.95039892,  0.65746416,
         0.4428871 , -0.2803352 , -0.92380733, -0.66305001, -0.03396529],
       [-0.34125599,  1.86956552, -1.59134194, -0.90872536,  0.05705716,
         0.21709507,  0.09480537,  0.13451712, -0.7442297 , -0.42976675],
       [ 1.9378714 ,  0.097539  , -2.81196642, -1.08207078,  1.34742912,
         0.11661226,  0.36237351,  0.60883722, -0.08073961,  1.4001608 ],
       [-1.28142125, -0.1357363 , -0.35808244,  0.26518023,  0.07072032,
        -0.19060895,  0.19608096,  1.00555752, -0.96253904,  1.43731603],
       [ 0.5337351 ,  0.10614917, -2.31842819, -0.94614727, -0.10993324,
         0.1236747 ,  0.27868214,  0.07382213, -0.55337285,  0.96185333]])

In [11]:
y_train[:5]

array([0, 0, 0, 0, 0])

## Model building

In [12]:
class NaiveBayes:

    def fit(self, X, y):                                                # for training the model
        n_samples, n_features = X.shape
        self.classes = np.unique(y)                                     # gives unique elements of y
        n_classes = len(self.classes)                                   # finds no of unique elements of y

        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)        # mean of the feature for the respective class
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)         # variance of the feature for the respective class
        self._priors = np.zeros((n_classes), dtype=np.float64)                  # 1D vector with size of no of classes

        for idx, each_class in enumerate(self.classes):
            X_c = X[y == each_class]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]                                  # calling _predict in list comprehension
        return np.array(y_pred)

    def _predict(self, x):                                                      # helper method
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self.classes):
            prior = np.log(self._priors[idx])                                   # log(P(Y))
            posterior = np.sum(np.log(self._pdf(idx, x)))                       # log(P(X1/Y)) + log(P(X2/Y)) + ....
            combined = prior + posterior
            posteriors.append(combined)

        # return class with highest posterior probability
        return self.classes[np.argmax(posteriors)]                              # argmax (log(P(X1/Y)) + log(P(X2/Y)) + .... + log(P(Y)))

    def _pdf(self, class_idx, x):                                               # gaussian formula
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator                                          # probability distribution equation

In [13]:
model = NaiveBayes()
model.fit(X_train, y_train)                                       # used to train model
y_pred = model.predict(X_test)

In [14]:
y_pred[:5]

array([1, 0, 0, 1, 1])

In [15]:
y_test[:5]

array([1, 0, 0, 1, 1])

## Evaluating test data

In [16]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_test == y_pred) / len(y_test)
    return accuracy

In [17]:
acc = accuracy(y_test, y_pred)
print("Accuracy:", acc*100)

Accuracy: 95.0
